In [8]:
import numpy as np

In [1]:
def read_input(filename, split = False, convert_to_int = False, sep = '\n'):
    f = open(filename)
    raw = f.read()[:-1]
    f.close()
    data = raw
    if split:
        data = data.split(sep)
    if convert_to_int:
        data = [int(item) for item in data]
    return data

In [2]:
sample_data, data = read_input('sample.txt',True), read_input('input.txt',True)

In [98]:
class ChitonsCavern:
    def __init__(self, data):
        self.map = np.ones((len(data)+2,len(data[0])+2))*(len(data)**2*10)
        for y in range(1,len(data)+1):
            for x in range(1,len(data[0])+1):
                self.map[y,x] = int(data[y-1][x-1])

        self.root = (1,1)
        self.current = self.root
        self.queue = []
        self.distances = {self.root: 0}
        self.distances_map = np.zeros(self.map.shape)
    
    def run(self):
        while self.current != (self.map.shape[0]-2, self.map.shape[1]-2):
            #calc distance to unvisited neighbours
            for (dy,dx) in [(1,0),(0,1),(-1,0),(0,-1)]:
                neighbour = (self.current[0] + dy, self.current[1] + dx)
                #add them to queue
                if neighbour not in self.distances:
                    self.distances[neighbour] = self.distances[self.current] + self.map[neighbour]
                    self.queue.append(neighbour)
            #take step to the cheapest
            next_dot = self.queue[0]
            for dot in self.queue:
                if self.distances[dot] < self.distances[next_dot]:
                    next_dot = dot
            self.current = next_dot
            self.queue.remove(next_dot)
        return self.distances[self.current]
            
    def map_distances(self):
        for dot in self.distances:
            self.distances_map[dot] = self.distances[dot]
        print(self.distances_map)


In [94]:
%%time
ChitonsCavern(sample_data).run() == 40

CPU times: user 1.18 ms, sys: 12 µs, total: 1.2 ms
Wall time: 1.19 ms


True

In [101]:
%%time
ChitonsCavern(data).run()

CPU times: user 636 ms, sys: 3.2 ms, total: 639 ms
Wall time: 638 ms


403.0

In [126]:
class LargeChitonsCavern(ChitonsCavern):
    def __init__(self, data, n = 5):
        self.map = np.ones((len(data)*n+2,len(data[0])*n+2))*((len(data)*n)**2*10)
        self.original_block = np.ones((len(data),len(data[0])))
        for y in range(len(data)):
            for x in range(len(data[0])):
                self.original_block[y,x] = int(data[y][x])
                
        for i in range(n):
            for j in range(n):
                for y in range(self.original_block.shape[0]):
                    for x in range(self.original_block.shape[1]):
                        self.map[1 + y + self.original_block.shape[0] * i,
                                 1 + x + self.original_block.shape[1] * j] = self.original_block[y,x] + 1 *(i+j)
        
        for y in range(1,self.map.shape[0]-1):
            for x in range(1,self.map.shape[1]-1):
                if self.map[y,x] > 9:
                    self.map[y,x] = self.map[y,x] % 9

        self.root = (1,1)
        self.current = self.root
        self.queue = []
        self.distances = {self.root: 0}
        self.distances_map = np.zeros(self.map.shape)        

In [127]:
%%time
LargeChitonsCavern(sample_data).run() == 315

CPU times: user 87.8 ms, sys: 2.54 ms, total: 90.4 ms
Wall time: 98.2 ms


True

In [128]:
%%time
LargeChitonsCavern(data).run()

CPU times: user 1min 25s, sys: 134 ms, total: 1min 26s
Wall time: 1min 26s


2840.0